In [1]:
import os

In [2]:
import pickle
import numpy as np
import random
import multiprocessing
from sklearn.manifold import TSNE

In [3]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

In [4]:
from scipy.stats import gaussian_kde

In [5]:
from scipy.misc import imsave

In [6]:
from collections import OrderedDict
CATEGORY_DICT = OrderedDict({
0 : "airplane", 
1: "automobile",
2: "bird",
3:"cat",
4:"deer",
5:"dog",
6:"frog",
7:"horse",
8:"ship",
9:"truck"
})

In [7]:
import math
def get_data(X, BATCH_SIZE):
    index = 0
    while True:
        if index + BATCH_SIZE <= len(X):
            data = X[index:index+BATCH_SIZE]
            index += BATCH_SIZE
        else:
            data = X[:BATCH_SIZE]
            index = BATCH_SIZE
        yield data


def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:]
    image = np.zeros((height*shape[0], width*shape[1], shape[2]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1], :] = img[:, :, :]
    return image

In [8]:
MODEL = "SNGAN"

In [9]:
import tensorflow as tf
session = tf.Session()

In [10]:
if MODEL != "VAE":
    MODEL == "SNGAN"
    imported_meta = tf.train.import_meta_graph("./model/model.ckpt-100000.meta")  
    imported_meta.restore(session, tf.train.latest_checkpoint('./model/'))
    image_input = session.graph.get_tensor_by_name("Placeholder_2:0")
    encoded = session.graph.get_tensor_by_name("critic_1/Reshape:0")
    critic_output = session.graph.get_tensor_by_name("Reshape_1:0")
else:
    imported_meta = tf.train.import_meta_graph("./VAE_model/model.ckpt-125000.meta")
    imported_meta.restore(session, tf.train.latest_checkpoint('./VAE_model/'))
    image_input = session.graph.get_tensor_by_name("Placeholder_1:0")
    encoded = session.graph.get_tensor_by_name("Encoder/flatten_1/Reshape:0")

INFO:tensorflow:Restoring parameters from ./model/model.ckpt-100000


In [11]:
def softplus(x):
    return np.log(1.0 + np.exp(x))

In [12]:
def get_encoded_representation(X):
    codes = []
    batches = len(X) // 64
    for i in range(batches):
        images = X[i*64:(i+1)*64]
        fd = {image_input: (images-127.5)/127.5}
        code = session.run(encoded, feed_dict=fd)
        codes.append(code)
    codes = np.concatenate(codes, axis = 0)
    return codes

In [13]:
def get_IMAGES_SOURCE_DIR(MODE):
    if MODE == "train":
        IMAGES_SOURCE_DIR = TRAIN_IMAGES_DIR
    elif MODE == "test":
        IMAGES_SOURCE_DIR = TEST_IMAGES_DIR
    elif MODE == "generated":
        IMAGES_SOURCE_DIR = GENERATED_IMAGES_DIR
    return IMAGES_SOURCE_DIR

In [14]:
BASE_DIR = os.getcwd()
STATS_DIR = os.path.join(BASE_DIR, "stats")
IMAGES_DIR = os.path.join(BASE_DIR, "TSNE_images")
PLOTS_DIR = os.path.join(BASE_DIR, "desktop_plots")
assert(os.path.exists(STATS_DIR))

In [15]:
TRAIN_IMAGES_DIR = os.path.join(IMAGES_DIR, "train")
TEST_IMAGES_DIR = os.path.join(IMAGES_DIR, "test")
GENERATED_IMAGES_DIR = os.path.join(IMAGES_DIR, "generated")
if not os.path.exists(IMAGES_DIR):
    os.mkdir(IMAGES_DIR)
    os.mkdir(TRAIN_IMAGES_DIR)
    os.mkdir(TEST_IMAGES_DIR)
    os.mkdir(GENERATED_IMAGES_DIR)
else:
    if not os.path.exists(TRAIN_IMAGES_DIR):
        os.mkdir(TRAIN_IMAGES_DIR)
    if not os.path.exists(TEST_IMAGES_DIR):
        os.mkdir(TEST_IMAGES_DIR)
    if not os.path.exists(GENERATED_IMAGES_DIR):
        os.mkdir(GENERATED_IMAGES_DIR)

In [16]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource, reset_output, output_notebook
from bokeh.transform import linear_cmap
from bokeh.palettes import d3
from bokeh.models import CategoricalColorMapper
reset_output()

TOOLTIPS = """
    <div>
        <div>
            <img
                src="@imgs" height="64" width="64"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div>
    </div>
"""

def plot(MODE, projection):
    IMAGES_SOURCE_DIR = get_IMAGES_SOURCE_DIR(MODE)
    if MODE != "generated":
        source = ColumnDataSource(data=dict(
            x=projection[:,0],
            y=projection[:,1],
            imgs=[os.path.join(IMAGES_SOURCE_DIR, str(i)+".png") for i in range(MAX_PROJECTIONS)],
            labels = [CATEGORY_DICT[i] for i in y]
        ))
        color_map = CategoricalColorMapper(factors=list(CATEGORY_DICT.values()),
                                       palette=d3['Category10'][10])
        color = {'field': 'labels', 'transform': color_map}
        legend = 'labels'
        p = figure(plot_width=1600, plot_height=800, tooltips=TOOLTIPS,
               title=MODE)
        p.circle('x', 'y', color=color, size=10, source=source, legend=legend)
    else:
        source = ColumnDataSource(data=dict(
            x=projection[:,0],
            y=projection[:,1],
            imgs=[os.path.join(IMAGES_SOURCE_DIR, str(i)+".png") for i in range(MAX_PROJECTIONS)]))
        color_map = None
        color = None
        legend = None
        p = figure(plot_width=1600, plot_height=800, tooltips=TOOLTIPS,
               title=MODE)
        p.circle('x', 'y', size=10, source=source)

    output_file(os.path.join(PLOTS_DIR, MODEL+"_"+MODE+"_tsne.html"))

    show(p)

In [17]:
'''
DICT = {"train":{}, "test":{}, "generated":{}}
MAX_PROJECTIONS = 64 * (600 // 64)
for MODE in ["generated", "test", "train"]:
    fname = os.path.join(STATS_DIR, MODE+"_outputs.pkl")
    with open(fname, "rb") as f:
        d = pickle.load(f)
    X = d["images"]
    y = np.array(d["labels"]) if d["labels"] is not None else None
    d_out = np.array(d["critic_outputs"])

    IMAGES_SOURCE_DIR = get_IMAGES_SOURCE_DIR(MODE)

    perm = np.random.permutation(len(X))
    DICT[MODE]["X"] = X[perm][:MAX_PROJECTIONS]
    DICT[MODE]["y"] = y[perm][:MAX_PROJECTIONS] if y is not None else None
    DICT[MODE]["d_out"] = d_out[perm][:MAX_PROJECTIONS]
    for i in range(len(DICT[MODE]["X"])):
        imsave(os.path.join(IMAGES_SOURCE_DIR, str(i)+".png"), DICT[MODE]["X"][i])
'''

'\nDICT = {"train":{}, "test":{}, "generated":{}}\nMAX_PROJECTIONS = 64 * (600 // 64)\nfor MODE in ["generated", "test", "train"]:\n    fname = os.path.join(STATS_DIR, MODE+"_outputs.pkl")\n    with open(fname, "rb") as f:\n        d = pickle.load(f)\n    X = d["images"]\n    y = np.array(d["labels"]) if d["labels"] is not None else None\n    d_out = np.array(d["critic_outputs"])\n\n    IMAGES_SOURCE_DIR = get_IMAGES_SOURCE_DIR(MODE)\n\n    perm = np.random.permutation(len(X))\n    DICT[MODE]["X"] = X[perm][:MAX_PROJECTIONS]\n    DICT[MODE]["y"] = y[perm][:MAX_PROJECTIONS] if y is not None else None\n    DICT[MODE]["d_out"] = d_out[perm][:MAX_PROJECTIONS]\n'

In [18]:
fname = os.path.join(STATS_DIR, "TSNE.pkl")
with open(fname, "rb") as f:
    DICT = pickle.load(f)

In [19]:
CODES = []
MAX_PROJECTIONS = 64 * (600 // 64)
for MODE in ["generated", "test", "train"]:

    X = DICT[MODE]['X']
    y = DICT[MODE]['y'] 
    d_out = DICT[MODE]['d_out']
    IMAGES_SOURCE_DIR = get_IMAGES_SOURCE_DIR(MODE)
    
    codes = get_encoded_representation(X)
    CODES.append(codes)
    tsne = TSNE()
    projections = tsne.fit_transform(codes)
    plot(MODE, projections)
CODES = np.concatenate(CODES, axis = 0)
tsne = TSNE()
PROJECTIONS = tsne.fit_transform(CODES)

/home/sudeep/.conda/envs/keras_gpu_tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:11: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/sudeep/.conda/envs/keras_gpu_tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:11: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/sudeep/.conda/envs/keras_gpu_tensorflow/lib/python3.6/site-packages/ipykernel/__main__.py:11: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [20]:
PROJECTIONS.shape

(1728, 2)

In [21]:
CODES.shape

(1728, 8192)

In [22]:
np.mean(np.max(CODES, axis=0)), np.mean(np.min(CODES, axis= 0))

(0.25192285, -0.0013653218)

In [23]:
source = ColumnDataSource(data=dict(
    x=PROJECTIONS[:,0],
    y=PROJECTIONS[:,1],
    imgs=[os.path.join(GENERATED_IMAGES_DIR, str(i)+".png") for i in range(MAX_PROJECTIONS)]\
    +[os.path.join(TEST_IMAGES_DIR, str(i)+".png") for i in range(MAX_PROJECTIONS)]\
    +[os.path.join(TRAIN_IMAGES_DIR, str(i)+".png") for i in range(MAX_PROJECTIONS)],
    labels = ["generated"]*(len(PROJECTIONS)//3) +  ["test"]*(len(PROJECTIONS)//3) \
    + ["train"]*(len(PROJECTIONS)//3)
    ))
color_map = CategoricalColorMapper(factors=["generated", "test", "train"], palette=d3['Category10'][3])
color = {'field': 'labels', 'transform': color_map}
legend = 'labels'
p = figure(plot_width=1600, plot_height=800, tooltips=TOOLTIPS, title="Combined projections")
p.circle('x', 'y', color=color, size=10, source=source, legend=legend)
output_file(os.path.join(PLOTS_DIR, MODEL+"_Combined_tsne.html"))
show(p)

----------------------